In [1]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# タイトルとカテゴリーのデータセットを読み込み
import pickle
import pandas as pd
drive_dir = "drive/My Drive/Colab Notebooks/livedoor_data/"

with open(drive_dir + "livedoor_datasets.pickle", 'rb') as f:
  livedoor_data = pickle.load(f)

In [31]:
import re
from tqdm import tqdm
import torchtext
from torchtext import data
from torchtext import datasets
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from itertools import chain
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

sentences = []
for body in tqdm(livedoor_data['body']):
  for text in body.split('\n'):
    if len(text) == 0: continue
    if not text[-1] == '。': continue
    text = re.sub(r'[!-/:-@[-`{-~■-♯]', r' ', text)
    text = re.sub(r'[︰-＠]', r'', text)
    text = re.sub(r'[\u3000]', r'', text)
    sentences.append(text)

100%|██████████| 7376/7376 [00:00<00:00, 19329.85it/s]


In [42]:
def analyzer(text):
  return list(text)

char_sentences = []
char_labels = []

char2index = {}
char2index.update({"<pad>":0})
for text in tqdm(sentences):
    char_list = analyzer(text)
    char_labels.append([1 for _ in range(len(char_list))])
    for ch in char_list:
        if ch in char2index: continue
        char2index[ch] = len(char2index)
    char_sentences.append([char2index[ch] for ch in char_list])
print("char size : ", len(char2index))

100%|██████████| 55684/55684 [00:02<00:00, 23643.37it/s]

char size :  3403


In [44]:
max_len = 0
for chars in char_sentences:
  _l = len(chars)
  if max_len < _l:
    max_len = _l
print('max length', max_len)

max length 1206


In [49]:
pad_char_sentences = []
pad_char_labels = []
for chars, labels in tqdm(zip(char_sentences, char_labels), total=len(char_sentences)):
  for i in range(max_len - len(chars)):
    chars.insert(0, 0) # 前パディング
    labels.insert(0, 0)
  pad_char_sentences.append(chars)
  pad_char_labels.append(labels)

100%|██████████| 55684/55684 [00:33<00:00, 1642.89it/s]


In [51]:
def train2batch(title, category, batch_size=100):
  title_batch = []
  category_batch = []
  title_shuffle, category_shuffle = shuffle(title, category)
  for i in range(0, len(title), batch_size):
    title_batch.append(title_shuffle[i:i+batch_size])
    category_batch.append(category_shuffle[i:i+batch_size])
  return title_batch, category_batch

In [52]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

